In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from torch.utils.tensorboard import SummaryWriter
import numpy as np
import gurobipy
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

import robust_value_approx.relu_system as relu_system
import robust_value_approx.lyapunov as lyapunov
import robust_value_approx.pybullet_data as pybullet_data

import robust_value_approx.test.train_2d_lyapunov_utils as train_2d_lyapunov_utils
import robust_value_approx.latent_system as latent_system

In [ ]:
# data
urdf = 'pendulum.urdf'
dtype = torch.float64
pybullet_x_lo = torch.tensor([-np.pi/2, -5.], dtype=dtype)
pybullet_x_up = torch.tensor([np.pi/2, 5.], dtype=dtype)
dt = .1
grayscale = True
image_width = 48
image_height = 48

# training
num_samples = 1000
validation_ratio = .01
batch_size = 20
dyn_loss_weight = 1.
lyap_loss_weight = 1.

# encoder-decoder
use_conv = False
use_bce = True
use_variational = False
z_dim = 3
z_lo = -1. * torch.ones(z_dim, dtype=dtype)
z_up = 1. * torch.ones(z_dim, dtype=dtype)

# dynamics nn
dyn_nn_width = 5
dyn_nn_depth = 1

# lyapunov nn
lyap_nn_width = 5
lyap_nn_depth = 1

In [ ]:
pbsg = pybullet_data.PybulletSampleGenerator(urdf, image_width=image_width, image_height=image_height, dtype=dtype)

In [ ]:
x_data, x_next_data, X_data, X_next_data = pbsg.generate_dataset(pybullet_x_lo, pybullet_x_up, dt, num_samples, grayscale=grayscale)

In [ ]:
i = np.random.choice(X_data.shape[0], 1)[0]
pybullet_data.show_sample(X_data[i,:], X_next_data[i,:])

In [ ]:
dyn_nn_layers = [torch.nn.Linear(z_dim, dyn_nn_width), torch.nn.ReLU()]
for i in range(dyn_nn_depth):
    dyn_nn_layers += [torch.nn.Linear(dyn_nn_width, dyn_nn_width), torch.nn.ReLU()]
dyn_nn_layers += [torch.nn.Linear(dyn_nn_width, z_dim)]
dyn_nn_model = torch.nn.Sequential(*dyn_nn_layers).type(dtype)

lyap_nn_layers = [torch.nn.Linear(z_dim, lyap_nn_width), torch.nn.ReLU()]
for i in range(lyap_nn_depth):
    lyap_nn_layers += [torch.nn.Linear(lyap_nn_width, lyap_nn_width), torch.nn.ReLU()]
lyap_nn_layers += [torch.nn.Linear(lyap_nn_width, 1)]
lyap_nn_model = torch.nn.Sequential(*lyap_nn_layers).type(dtype)

## Learning in image space

In [ ]:
X_dataset = TensorDataset(X_data.double(), X_next_data.double())
# X_dataset = TensorDataset(X_data[:5,:].double(), X_next_data[:5,:].double())

train_size = int((1. - validation_ratio) * len(X_dataset))
test_size = len(X_dataset) - train_size
train_dataset_img, validation_dataset_img = torch.utils.data.random_split(X_dataset, [train_size, test_size])
train_dataloader_img = DataLoader(
    train_dataset_img,
    batch_size=batch_size,
)
validation_dataloader_img = DataLoader(
    validation_dataset_img,
    batch_size=len(validation_dataset_img),
)
validation_data_img = validation_dataloader_img.__iter__().next()

In [ ]:
relu_sys = relu_system.AutonomousReLUSystem(dtype, z_lo, z_up, dyn_nn_model)
lyap = lyapunov.LyapunovDiscreteTimeHybridSystem(relu_sys, lyap_nn_model)
latent_sys = latent_system.LatentAutonomousReLUSystem(relu_sys, lyap, use_conv=use_conv, use_bce=use_bce, use_variational=use_variational,
                                                      image_width=image_width, image_height=image_height, grayscale=grayscale)

In [ ]:
i = np.random.choice(X_data.shape[0], 1)[0]
x_traj = latent_sys.rollout(X_data[i, :], 5, clamp=True)
for n in range(x_traj.shape[0]):
    pybullet_data.show_sample(x_traj[n, :], clamp=True)

In [ ]:
# (for 2D latent systems only)
fig = plt.figure()
ax = fig.gca(projection='3d')
train_2d_lyapunov_utils.plot_lyapunov(ax, latent_sys.lyapunov.lyapunov_relu,
                                      latent_sys.V_lambda, latent_sys.z_equilibrium,
                                      latent_sys.z_lo, latent_sys.z_up, [10, 10], 10)

In [ ]:
optimizer = torch.optim.Adam([{'params': latent_sys.encoder.parameters()},
                              {'params': latent_sys.decoder.parameters()},
                              {'params': latent_sys.lyapunov.lyapunov_relu.parameters()}])

writer = SummaryWriter()
n_iter = 0

In [ ]:
for epoch_i in range(1000):
    for x, x_next in train_dataloader_img:
        optimizer.zero_grad()
        
        if dyn_loss_weight > 0:
            x_decoded, x_next_pred_decoded, z_mu, z_log_var = latent_sys.vae_forward(x)
            dyn_loss = latent_sys.vae_loss(x, x_next, x_decoded, x_next_pred_decoded, z_mu, z_log_var)
        else:
            dyn_loss = torch.tensor(0., dtype=latent_sys.dtype)
        
        if lyap_loss_weight > 0:
            lyap_loss = latent_sys.lyapunov_loss()
        else:
            lyap_loss = torch.tensor(0., dtype=latent_sys.dtype)
        
        loss = dyn_loss_weight * dyn_loss + lyap_loss_weight * lyap_loss
        
        loss.backward()
        optimizer.step()
        
        writer.add_scalar('Dynamics/train', dyn_loss_weight * dyn_loss.item(), n_iter)
        writer.add_scalar('Lyapunov', lyap_loss_weight * lyap_loss.item(), n_iter)
        n_iter += 1
      
    with torch.no_grad():
        x_decoded, x_next_pred_decoded, z_mu, z_log_var = latent_sys.vae_forward(validation_data_img[0])
        validation_loss = latent_sys.vae_loss(validation_data_img[0], validation_data_img[1], x_decoded, x_next_pred_decoded, z_mu, z_log_var)
        writer.add_scalar('Dynamics/validation', validation_loss.item(), n_iter)

## Learning in state space

In [ ]:
x_dataset = TensorDataset(x_data, x_next_data)
train_size = int((1. - validation_ratio) * len(x_dataset))
test_size = len(x_dataset) - train_size
train_dataset, validation_dataset = torch.utils.data.random_split(x_dataset, [train_size, test_size])
train_dataloader = DataLoader(
    train_dataset,
    batch_size=batch_size,
)
validation_dataloader = DataLoader(
    validation_dataset,
    batch_size=len(validation_dataset),
)
validation_data = validation_dataloader.__iter__().next()